In [4]:
from yucca.pipeline.managers.YuccaManager import YuccaManager

Set some variables that we'll need

In [5]:
plans_name = "demo"

Create a manager (can be replaced by your own training script/class)

In [6]:
Manager = YuccaManager(
    enable_logging=False,
    ckpt_path=None,
    model_name="TinyUNet",
    model_dimensions="2D",
    num_workers=4,
    split_idx=0,
    step_logging=True,
    task="Task001_OASIS",
    planner=plans_name,
    patch_size=(32, 32),
    batch_size=2,
    max_epochs=2,
    val_batches_per_step=2,
    train_batches_per_step=2,
    accelerator="cpu",
)
Manager.run_training()

Seed set to 802151415
INFO:root:Loading plans.json
INFO:root:Getting patch size based on manual input of: (32, 32)
INFO:root:Using batch size: 2 and patch size: (32, 32)
INFO:root:YuccaLightningModule initialized with the following config: {'continue_from_most_recent': True, 'experiment': 'default', 'manager_name': 'YuccaManager', 'model_dimensions': '2D', 'model_name': 'TinyUNet', 'patch_based_training': True, 'planner_name': 'demo', 'task': 'Task001_OASIS', 'split_idx': 0, 'split_method': 'kfold', 'split_param': 5, 'plans_path': '/Users/zcr545/Desktop/Projects/repos/yucca_data/preprocessed/Task001_OASIS/demo/demo_plans.json', 'save_dir': '/Users/zcr545/Desktop/Projects/repos/yucca_data/models/Task001_OASIS/TinyUNet__2D/YuccaManager__demo/default/kfold_5_fold_0', 'train_data_dir': '/Users/zcr545/Desktop/Projects/repos/yucca_data/preprocessed/Task001_OASIS/demo', 'version_dir': '/Users/zcr545/Desktop/Projects/repos/yucca_data/models/Task001_OASIS/TinyUNet__2D/YuccaManager__demo/default

Composing Transforms
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/zcr545/miniconda3/envs/yuccaenv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 123.41it/s]torch.Size([2, 3, 32, 32]) torch.Size([2, 32, 32])
                                                                            

INFO:root:Starting training with data from: /Users/zcr545/Desktop/Projects/repos/yucca_data/preprocessed/Task001_OASIS/demo


/Users/zcr545/miniconda3/envs/yuccaenv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0: 100%|██████████| 2/2 [00:03<00:00,  0.52it/s, v_num=0]torch.Size([2, 3, 32, 32]) torch.Size([2, 32, 32])
torch.Size([2, 3, 32, 32]) torch.Size([2, 32, 32])
Epoch 1: 100%|██████████| 2/2 [00:15<00:00,  0.13it/s, v_num=0]torch.Size([2, 3, 32, 32]) torch.Size([2, 32, 32])
torch.Size([2, 3, 32, 32]) torch.Size([2, 32, 32])
Epoch 1: 100%|██████████| 2/2 [00:35<00:00,  0.06it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 2/2 [00:35<00:00,  0.06it/s, v_num=0]
